라이브러리 import 및 셋팅

In [ ]:
!pip install -U textblob
!python -m textblob.download_corpora
!pip install bs4

In [ ]:
import os
import pandas as pd
import re
import string
import nltk

from bs4 import BeautifulSoup as bs
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

코로나 시기별 파일 불러오기

In [ ]:
def load_files():

  path = '/content/drive/MyDrive/졸프/youtuberdata/DavidCommentList/'

  files = []

  for r, d, f in os.walk(path):
      for file in f:
          if '.csv' and 'T3' in str(file):
            
            files.append(os.path.join(r, file))
  
  return files
  


In [ ]:
paths = load_files()

In [ ]:
paths

결과 저장할 리스트들

In [ ]:
raw_comment_list = []
clean_comment_list = []
polarity_list = []
subjectivity_list = []
sentiment_list = []
emotion_list = []

TimeCode가 같은 파일 한번에 불러오기.

raw_comment_list로 모든 정제 전 코멘트 불러오기

In [ ]:
for path in paths:
  youtube_data = pd.read_csv(path,encoding = 'utf-8-sig', engine='python')
  for comment in youtube_data['comment']:
    raw_comment_list.append(comment)

In [ ]:
len(raw_comment_list)

raw_comment 텍스트 전처리 함수

In [ ]:
def preprocessing(review, remove_stopwords = False ): 
    review_text = bs(review, "html5lib").get_text()	

    review_text = re.sub("[^a-zA-Z]", " ", review_text)

    words = review_text.lower().split()

    if remove_stopwords: 
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        clean_review = ' '.join(words)
    else:
        clean_review = ' '.join(words)

    return clean_review

In [ ]:
def cleaning_comment(data):
  clean_comments = []
  for comment in data:
      clean_comments.append(preprocessing(comment,remove_stopwords=True))
  return clean_comments

In [ ]:
clean_comment_list = cleaning_comment(raw_comment_list)

TextBlob을 이용한 감성분석

In [ ]:
def sentiment_analysis(clean_comment):
  result=TextBlob(clean_comment)
  polarity_list.append(result.polarity)
  subjectivity_list.append(result.subjectivity)

In [ ]:
len(polarity_list)

In [ ]:
def process_sentiment():
  for sentiment in polarity_list:
    if sentiment>= 0.5 and sentiment<=1.0:
	    sentiment_list.append("strongPositive")
    elif sentiment>0.0 and sentiment<0.5:
      sentiment_list.append("positive")
    elif sentiment == 0:
      sentiment_list.append("neutral")
    elif sentiment <0 and sentiment>= -0.5:
      sentiment_list.append("negative")
    elif sentiment < -0.5:
      sentiment_list.append("strongNegative")

Emotion분석

In [ ]:
emotion_file = pd.read_csv('/content/drive/MyDrive/졸프/NRC-Emotion-Lexicon-clean.csv',encoding = 'utf-8-sig')
emotion_file.info

In [ ]:
def flat_list(array): 
    a=[]
    for i in array:
        if type(i) == type(list()):
            a+=(flat_list(i))       
        else:
            a.append(i)
    return a

In [ ]:
def extract_emotion_list(text,file):

  lower_case = text.lower()

  cleaned_text = lower_case.translate(str.maketrans('', '', string.punctuation))

  tokenized_words = word_tokenize(cleaned_text, "english")

  lemma_words = []
  
  for word in tokenized_words:
      word = WordNetLemmatizer().lemmatize(word)
      lemma_words.append(word)

  emotion_lists = []

  for j in range(len(file['wordlist'])):
      word = file['wordlist'][j]

      if word in lemma_words:
          emotion_lists.append(file['emotions'][j].split(':'))
  emotion_lists = flat_list(emotion_lists)
  return emotion_lists



In [ ]:
for i in range(len(clean_comment_list)):
  if i%1000 == 0:
    print(i)
  sentiment_analysis(clean_comment_list[i])
  emotion_list.append(extract_emotion_list(clean_comment_list[i],emotion_file))
process_sentiment()

sentiment analysis결과와 emotion analysis결과 한 파일에 저장 

In [ ]:
df = pd.DataFrame({'raw_comment':raw_comment_list, 'clean_comment':clean_comment_list,
                   'polarity':polarity_list,'subjectivity':subjectivity_list,
                   'sentiment':sentiment_list,'emotion_list':emotion_list})

In [ ]:
df.to_csv("/content/drive/MyDrive/졸프/감성분석result/David_Result/T3_David_Comment_result.csv",encoding = 'utf-8-sig')

In [ ]:
   df